In [50]:
%reload_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
from utils.dummy import dummy_data
from utils.dummy import dummy_doi
from utils.dummy import get_dummy_info
from utils.utils import get_paper_citation_pairs
from utils.utils import get_fully_connected_network
from acm_api.acm import acm_meta
from acm_api.acm import acm_db_init
from evaluation.evaluate import Train_Test_Split_Plain
import matplotlib.pyplot as plt
from cf.cf import CF
from cf.cf_unit import CFU
from cf.cf_pr import CFUPR
from cf.cf_full_hits import CFUH
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords

<h2>Processing the input file with doi of papers</h2>

In [51]:
doi_list, data, data_citedby = acm_db_init()

processing doi....


  0%|          | 0/6 [00:00<?, ?it/s]

extracting citation's doi....


  0%|          | 0/155 [00:00<?, ?it/s]

extracting cited_by doi....


...

  0%|          | 0/18 [00:00<?, ?it/s]

extracting cited_by doi....


  0%|          | 0/4 [00:00<?, ?it/s]

extracting citation's doi....


  0%|          | 0/136 [00:00<?, ?it/s]

In [52]:
# Paper's doi and it's citations
data

{'10.1145/3439723': ['10.1109/CVPR.2014.487',
  '10.1145/1531326.1531330',
  '10.1137/S0363012995293694',
  '10.1007/BF01584320',
  '10.5555/2188385.2188410',
  '10.5555/3295222.3295327',
  '10.1145/3301282',
  '10.1145/2601097.2601122',
  '10.1145/3072959.3073659',
  '10.1016/j.patcog.2012.06.021',
  '10.1109/AVSS.2009.58',
  '10.1145/3077136.3080842',
  '10.1023/A%3A1026543900054',
  '10.5555/3294996.3295059',
  '10.1609/aaai.v33i01.33018901'],
 '10.1145/3446374': ['10.1162/neco.2006.18.7.1527',
  '10.1109/access.2019.2905015',
  '10.1007/s11263-019-01265-2',
  '10.1006/jcss.1997.1504',
  '10.1023/A%3A1026543900054',
  '10.1162/089976698300017746',
  '10.1145/3209978.3210184'],
 '10.1145/3459992': ['10.1145/2976749.2978318',
  '10.1504/IJSN.2015.071829',
  '10.1109/TIFS.2019.2891116',
  '10.1145/3374751',
  '10.1007/11681878_14',
  '10.1109/SECURWARE.2009.48',
  '10.1109/TPAMI.2010.77',
  '10.1145/2810103.2813677',
  '10.1162/089976600300015015',
  '10.5555/3295222.3295327',
  '10.11

In [4]:
# data_citedby

<h2>Code for creating the rating matrix</h2>

In [5]:
# Store the citation space
citation_set = set()

# Store the citation space including the cited_by space
citation_set_with_cited_by = set()

# Here we are capturing the all the citation for a given paper
# Later we can use this to know the citation space
for key in data.keys():
    citation_set.update(data[key])
    citation_set_with_cited_by.update(data[key])
    
# We also need to include the queryset itself because they are cited by other papers    
citation_set_with_cited_by.update(data.keys())

# Index of the data without the cited_by key
index_ = data.keys()
                                                   
# Index of the data including the cited_by key
index_with_citedby = set(data.keys())
index_with_citedby.update(data_citedby.keys())
index_with_citedby = list(index_with_citedby)

# This dataframe does not include the cited_by data
df = pd.DataFrame(np.zeros((len(index_), len(citation_set))), index=index_, columns=citation_set)

# This dataframe includes the cited_by data
df_with_cited_by = pd.DataFrame(np.zeros((len(index_with_citedby), len(citation_set_with_cited_by))), \
                                index=index_with_citedby, columns=citation_set_with_cited_by)

for i in index_:
    for j in citation_set:
        if j in data[i]:
            df.loc[i][j] = 1


            
# Here data_citedby is the key-value with key being the paper
# that cites our query set and value is one of the paper in the
# query set
# Similary in data, the key is each paper in query set and
# value is list of citations
overall_ = {**data_citedby, **data}
overall = get_fully_connected_network(overall_, data_citedby.keys(), acm=False)
for i in index_with_citedby:
    for j in citation_set_with_cited_by:
        if j in overall[i]:
            df_with_cited_by.loc[i][j] = 1

df.to_csv('citation-web/matrix-way/citation-matrix.csv')
df_with_cited_by.to_csv('citation-web/matrix-way/citation-full-matrix.csv')

  0%|          | 0/220 [00:00<?, ?it/s]

<h2>Assign root nodes that can be used for cf full version</h2>

In [6]:
root_nodes = df.index.values

<h2>Creating the citation pair</h2>

In [7]:
# This only includes the citation data
citation_pair_path = get_paper_citation_pairs(df)

# This also includes the cited_by data 
citation_pair_full_path = get_paper_citation_pairs(df_with_cited_by, out_file='citation-full-pairs.txt')

writing citation pairs....


  0%|          | 0/20 [00:00<?, ?it/s]

writing citation pairs....


  0%|          | 0/220 [00:00<?, ?it/s]

<h2>Below are the experiments</h2>

In [8]:
evaluate = Train_Test_Split_Plain(CFU, min_ref_limit=5, train_size=0.5)
evaluate.fit(data)

The following doi's are ignored because of low reference :
{}
writing citation pairs....


  0%|          | 0/20 [00:00<?, ?it/s]

removing the indexes with no citations....
normalizing dataframe....


100%|█████████████████████████████████████████| 20/20 [00:00<00:00, 2838.50it/s]


creating similarity matrix....


...

computing recommendations for : 1


0it [00:00, ?it/s]


finishing evaluation....


  0%|          | 0/10 [00:00<?, ?it/s]

0.0

In [9]:
evaluate = Train_Test_Split_Plain(CF, min_ref_limit=5, train_size=0.5)
evaluate.fit(data)

The following doi's are ignored because of low reference :
{}
writing citation pairs....


  0%|          | 0/20 [00:00<?, ?it/s]

Creating similarity matrix....


100%|█████████████████████████████████████████| 109/109 [00:01<00:00, 77.58it/s]


Computing recommendations for : 9


...

Computing recommendations for : 1


0it [00:00, ?it/s]


finishing evaluation....


  0%|          | 0/10 [00:00<?, ?it/s]

0.0

In [10]:
evaluate = Train_Test_Split_Plain(CFUPR, min_ref_limit=5, train_size=0.5, normalize_similarity=True)
evaluate.fit(data)

The following doi's are ignored because of low reference :
{}
writing citation pairs....


  0%|          | 0/20 [00:00<?, ?it/s]

removing the indexes with no citations....
page rank normalization on dataframe....


100%|█████████████████████████████████████████| 20/20 [00:00<00:00, 5485.62it/s]


creating similarity matrix....


...

Computing recommendations for : 1


100%|████████████████████████████████████████████| 2/2 [00:00<00:00, 179.87it/s]

finishing evaluation....


  0%|          | 0/10 [00:00<?, ?it/s]

0.2995631807125634

In [14]:
evaluate = Train_Test_Split_Plain(CFUH, min_ref_limit=5, train_size=0.5, normalize_similarity=True, root_node_indexes=root_nodes)
overall = {**data, **data_citedby}
evaluate.fit(overall)

The following doi's are ignored because of low reference :
{'204': 4,
 '205': 3,
 '208': 3,
 '212': 3,
 '223': 4,
 '228': 4,
 '238': 4,
 '242': 4,
 '265': 3,
 '266': 4,
 '277': 4,
 '279': 4,
 '290': 4}
writing citation pairs....


  0%|          | 0/207 [00:00<?, ?it/s]

removing the indexes with no citations....
Deleting :  298
Deleting :  289
Deleting :  218
Deleting :  262
Deleting :  256
Deleting :  284
Deleting :  255
Deleting :  280
Deleting :  251
Deleting :  231
page rank normalization on dataframe....


100%|███████████████████████████████████████| 197/197 [00:00<00:00, 5548.69it/s]


creating similarity matrix....


...

['1' '2' '3' '4' '5' '6' '7' '8' '9' '10' '11' '12' '13' '14' '15' '16'
 '17' '18' '19' '20' '432' '488' '486' '484' '419' '438' '454' '418' '410'
 '490' '477' '423' '412' '442' '406' '460' '447' '437' '449' '482' '499'
 '448' '408' '453' '497' '461' '434' '491' '498' '480' '471' '422' '407'
 '439' '405' '462' '445' '466' '436' '473' '409' '451' '468' '430' '431'
 '464' '470' '400' '426' '416' '493' '203' '200' '202' '495' '443' '248'
 '209' '297' '226' '245' '414' '234' '269' '479' '273' '210' '270' '452'
 '283' '217' '403' '465' '292' '278' '230' '249' '216' '220' '429' '207'
 '219' '233' '411' '250' '469' '232' '474' '281' '254' '287' '247' '263'
 '417' '288' '299' '296' '475' '494' '458' '275' '222' '404' '476' '467'
 '463' '291' '496' '450' '295' '487' '440' '481' '261' '294' '472' '267'
 '227' '415' '206' '241' '489' '224' '214' '282' '243' '244' '215' '235'
 '246' '274' '253' '402' '229' '433' '420' '435' '268' '264' '259' '444'
 '272' '258' '237' '485' '211' '285' '401' '201' '

100%|██████████████████████████████████████████| 13/13 [00:00<00:00, 189.18it/s]

finishing evaluation....


  0%|          | 0/94 [00:00<?, ?it/s]

0.21366380205976676

In [1]:
doc1 = "Generative adversarial networks (GANs) have been extensively studied in the past few years. Arguably their most significant impact has been in the area of computer vision where great advances have been made in challenges such as plausible image generation, image-to-image translation, facial attribute manipulation, and similar domains. Despite the significant successes achieved to date, applying GANs to real-world problems still poses significant challenges, three of which we focus on here. These are as follows: (1) the generation of high quality images, (2) diversity of image generation, and (3) stabilizing training. Focusing on the degree to which popular GAN technologies have made progress against these challenges, we provide a detailed review of the state-of-the-art in GAN-related research in the published scientific literature. We further structure this review through a convenient taxonomy we have adopted based on variations in GAN architectures and loss functions. While several reviews for GANs have been presented to date, none have considered the status of this field based on their progress toward addressing practical challenges relevant to computer vision. Accordingly, we review and critically discuss the most popular architecture-variant, and loss-variant GANs, for tackling these challenges. Our objective is to provide an overview as well as a critical analysis of the status of GAN research in terms of relevant progress toward critical computer vision application requirements. As we do this we also discuss the most compelling applications in computer vision in which GANs have demonstrated considerable success along with some suggestions for future research directions."
docs = {
    'doc2' : "Generative Adversarial Networks (GANs) is a novel class of deep generative models that has recently gained significant attention. GANs learn complex and high-dimensional distributions implicitly over images, audio, and data. However, there exist major challenges in training of GANs, mode collapse, non-convergence, and instability, due to inappropriate design of network architectre, use of objective function, and selection of optimization algorithm. Recently, to address these challenges, several solutions for better design and optimization of GANs have been investigated based on techniques of re-engineered network architectures, new objective functions, and alternative optimization algorithms. To the best of our knowledge, there is no existing survey that has particularly focused on the broad and systematic developments of these solutions. In this study, we perform a comprehensive survey of the advancements in GANs design and optimization solutions proposed to handle GANs challenges. We first identify key research issues within each design and optimization technique and then propose a new taxonomy to structure solutions by key research issues. In accordance with the taxonomy, we provide a detailed discussion on different GANs variants proposed within each solution and their relationships. Finally, based on the insights gained, we present promising research directions in this rapidly growing field.",
    'doc3' : "Generative Adversarial Networks (GANs) have promoted a variety of applications in computer vision and natural language processing among others, due to its generative model’s compelling ability to generate realistic examples plausibly drawn from an existing distribution of samples. GAN not only provides impressive performance on data generation-based tasks but also stimulates fertilization for privacy and security oriented research because of its game theoretic optimization strategy. Unfortunately, there are no comprehensive surveys on GAN in privacy and security, which motivates this survey to summarize systematically. The existing works are classified into proper categories based on privacy and security functions, and this survey conducts a comprehensive analysis of their advantages and drawbacks. Considering that GAN in privacy and security is still at a very initial stage and has imposed unique challenges that are yet to be well addressed this article also sheds light on some potential privacy and security applications with GAN and elaborates on some future research directions.",
    'doc4' : "Generative Adversarial Networks (GANs) have received wide attention in the machine learning field for their potential to learn high-dimensional, complex real data distribution. Specifically, they do not rely on any assumptions about the distribution and can generate real-like samples from latent space in a simple manner. This powerful property allows GANs to be applied to various applications such as image synthesis, image attribute editing, image translation, domain adaptation, and other academic fields. In this article, we discuss the details of GANs for those readers who are familiar with, but do not comprehend GANs deeply or who wish to view GANs from various perspectives. In addition, we explain how GANs operates and the fundamental meaning of various objective functions that have been suggested recently. We then focus on how the GAN can be combined with an autoencoder framework. Finally, we enumerate the GAN variants that are applied to various tasks and other fields for those who are interested in exploiting GANs for their research.",
    'doc5' : "Image denoising is a challenging task which aims to remove additional noise and preserve all useful information. Many existing image denoising algorithms focus on improving the typical object measure, peak signal-to-noise ratio (PSNR), and take the mean square error (MSE) as their loss function to train their networks. Although these algorithms can effectively improve the PSNR on the benchmark dataset, their denoised images often lose some important details or become over-smooth in some texture-rich regions. In order to solve this problem, we introduce Generative Adversarial Networks (GAN) and perceptual loss from single image super-resolution (SISR) field into our image denoising work. The GAN and perceptual loss can help our network to better focus on the recovering of details during denoising. To understand easily, we use the term Detail Loss to represent the whole loss which includes the MSE and the perceptual loss. Besides, we propose a new convolutional neural network which achieves state-of-the-art result on PSNR. Our experimental results show that our method outperforms the current state-of-the-art methods on preserving the details during denoising. Compared with the current state-of-the-art methods, the denoised images by our method are clearer, sharper and more realistic on details.",
    'doc6' : "With the increasing interest in the content creation field in multiple sectors such as media, education, and entertainment, there is an increased trend in the papers that use AI algorithms to generate content such as images, videos, audio, and text. Generative Adversarial Networks (GANs) is one of the promising models that synthesizes data samples that are similar to real data samples. While the variations of GANs models in general have been covered to some extent in several survey papers, to the best of our knowledge, this is the first paper that reviews the state-of-the-art video GANs models. This paper first categorizes GANs review papers into general GANs review papers, image GANs review papers, and special field GANs review papers such as anomaly detection, medical imaging, or cybersecurity. The paper then summarizes the main improvements in GANs that are not necessarily applied in the video domain in the first run but have been adopted in multiple video GANs variations. Then, a comprehensive review of video GANs models are provided under two main divisions based on existence of a condition. The conditional models are then further classified according to the provided condition into audio, text, video, and image. The paper concludes with the main challenges and limitations of the current video GANs models.",
    'doc7' : "The discriminative model learning for image denoising has been recently attracting considerable attentions due to its favorable denoising performance. In this paper, we take one step forward by investigating the construction of feed-forward denoising convolutional neural networks (DnCNNs) to embrace the progress in very deep architecture, learning algorithm, and regularization method into image denoising. Specifically, residual learning and batch normalization are utilized to speed up the training process as well as boost the denoising performance. Different from the existing discriminative denoising models which usually train a specific model for additive white Gaussian noise at a certain noise level, our DnCNN model is able to handle Gaussian denoising with unknown noise level (i.e., blind Gaussian denoising). With the residual learning strategy, DnCNN implicitly removes the latent clean image in the hidden layers. This property motivates us to train a single DnCNN model to tackle with several general image denoising tasks, such as Gaussian denoising, single image super-resolution, and JPEG image deblocking. Our extensive experiments demonstrate that our DnCNN model can not only exhibit high effectiveness in several general image denoising tasks, but also be efficiently implemented by benefiting from GPU computing.",
    'doc8' : "Image restoration tasks are ill-posed problems, typically solved with priors. Since the optimal prior is the exact unknown density of natural images, actual priors are only approximate and typically restricted to small patches. This raises several questions: How much may we hope to improve current restoration results with future sophisticated algorithms? And more fundamentally, even with perfect knowledge of natural image statistics, what is the inherent ambiguity of the problem? In addition, since most current methods are limited to finite support patches or kernels, what is the relation between the patch complexity of natural images, patch size, and restoration errors? Focusing on image denoising, we make several contributions. First, in light of computational constraints, we study the relation between denoising gain and sample size requirements in a non parametric approach. We present a law of diminishing return, namely that with increasing patch size, rare patches not only require a much larger dataset, but also gain little from it. This result suggests novel adaptive variable-sized patch schemes for denoising. Second, we study absolute denoising limits, regardless of the algorithm used, and the converge rate to them as a function of patch size. Scale invariance of natural images plays a key role here and implies both a strictly positive lower bound on denoising and a power law convergence. Extrapolating this parametric law gives a ballpark estimate of the best achievable denoising, suggesting that some improvement, although modest, is still possible.",
    'doc9' : "Image restoration is a long-standing problem in low-level computer vision with many interesting applications. We describe a flexible learning framework based on the concept of nonlinear reaction diffusion models for various image restoration problems. By embodying recent improvements in nonlinear diffusion models, we propose a dynamic nonlinear reaction diffusion model with time-dependent parameters (i.e., linear filters and influence functions). In contrast to previous nonlinear diffusion models, all the parameters, including the filters and the influence functions, are simultaneously learned from training data through a loss based approach. We call this approach TNRD—Trainable Nonlinear Reaction Diffusion. The TNRD approach is applicable for a variety of image restoration tasks by incorporating appropriate reaction force. We demonstrate its capabilities with three representative applications, Gaussian image denoising, single image super resolution and JPEG deblocking. Experiments show that our trained nonlinear diffusion models largely benefit from the training of the parameters and finally lead to the best reported performance on common test datasets for the tested applications. Our trained models preserve the structural simplicity of diffusion models and take only a small number of diffusion steps, thus are highly efficient. Moreover, they are also well-suited for parallel computation on GPUs, which makes the inference procedure extremely fast.",
}



In [13]:
%reload_ext autoreload
%autoreload 2
from acm_api.acm import download_search_acm

p, q, r, w = download_search_acm('generative+adversarial+networks+audio+and+video')

downloading paper meta data....
processing doi....


  0%|          | 0/278 [00:00<?, ?it/s]

extracting citation's doi....


  0%|          | 0/136 [00:00<?, ?it/s]

extracting citation's doi....


...

  0%|          | 0/2 [00:00<?, ?it/s]

extracting citation's doi....


  0%|          | 0/21 [00:00<?, ?it/s]

extracting citation's doi....


  0%|          | 0/40 [00:00<?, ?it/s]

In [14]:
%reload_ext autoreload
%autoreload 2

from tqdm import tqdm
# import tqdm.notebook as tqdm
import pandas as pd
import numpy as np
from acm_api.acm import download_search_acm
import json
import sqlite3
from wp.bm25 import bm25_score
from wp.bm25 import __spl_chars_removal__
from wp.bm25 import __stopwords_removal__
from wp.bm25 import __stemmer__
from wp.bm25 import __stemmer__
import re
import nltk
import pandas as pd
from nltk.stem import PorterStemmer
from rank_bm25 import *
from nltk.tokenize import word_tokenize
from gensim.parsing.preprocessing import STOPWORDS

In [15]:
dois = w
dois = [doi.replace('https://doi.org/','').strip() for doi in dois]

In [40]:
def __searchDB__(doi):
		"""
		This method is used to retrive the doi meta info
		from the db

		We assume that the doi record is already in the
		database
		"""
		con = sqlite3.connect('data.db')
		cur = con.cursor()
		cursor = con.execute("SELECT * FROM PAPERS WHERE doi=?;", (doi,))
		fetch_data = cursor.fetchone()
		return json.loads(fetch_data[1])

In [44]:
def __meta_2_doc__(meta):
		"""
		This method is used to produce the document/string out
		of a meta data
		"""
		str_ = meta['title'] + ' ' + meta['abstract']
		return str_

In [18]:
def __active__():
		"""
		This method is used to return the active paper
		string i.e., title + abstract
		"""
		dois = []
		temp = None

		with open('doi/doi.txt', 'r') as f:
			temp = f.readlines()

		for line in temp:
			dois.append(line.strip())

		full_str = ''

		for doi in dois:
			full_str += __meta_2_doc__(__searchDB__(doi)) + ' '

		return full_str, dois

In [19]:
scores = None
docs = {}
active_paper, active_dois = __active__()

for doi in dois:
    if doi not in active_dois:
        docs[doi] = __meta_2_doc__(__searchDB__(doi))

KeyError: 'abstract'

In [ ]:
bm25_score(active_paper, docs)

# tokenized_query = __stemmer__(__stopwords_removal__(__spl_chars_removal__(active_paper)))
# tokenized_corpus = [__stemmer__(__stopwords_removal__(__spl_chars_removal__(docs[key]))) for key in docs.keys()]

bm25 = BM25Okapi(tokenized_corpus)
doc_scores = bm25.get_scores(tokenized_query)

In [36]:

doc_scores

array([ 9.624919  , 31.6955819 , 21.63516912, 16.77741282, 17.28056942,
       12.55764823, 13.00952855, 14.39569889, 34.51419109, 13.3440492 ,
       10.68592579, 17.09416779, 14.87579323, 13.38076775, 36.02805349,
       14.71613408, 11.99611973, 18.79339695, 16.7131861 , 10.89178256,
       11.75276833, 10.44492566, 15.8613499 , 15.36407486, 11.59073502,
       16.30089809, 10.76727544, 16.28248213, 10.97561626, 14.30863173,
       19.55860238,  5.51509908, 23.32517799, 15.34401434, 18.54794759,
       17.76149599, 15.25936358, 15.18697128, 14.88193796, 15.5021622 ,
       18.12265484, 11.16296395, 15.27501289, 18.62694033, 16.47965517,
       25.26714673, 18.80524949])

In [34]:
tokenized_query = tokenized_query[:50]
tokenized_corpus = tokenized_corpus[:50]

In [23]:
obj = __searchDB__('10.1145/3487891')

In [22]:
import utils.utils as utils

In [25]:
utils.get_citation_doi(obj)

extracting citation's doi....


  0%|          | 0/136 [00:00<?, ?it/s]

['10.5555/2969033.2969125',
 '10.5555/3157096.3157165',
 '10.5555/3157096.3157340',
 '10.5555/3305890.3305954',
 '10.5555/3367243.3367316',
 '10.1609/aaai.v33i01.33019299',
 '10.1145/3123266.3127905',
 '10.1007/978-3-030-58517-4_31',
 '10.1007/978-3-030-00928-1_60',
 '10.1109/TPAMI.2013.248',
 '10.1109/ICCV.2005.28',
 '10.5555/3157096.3157346',
 '10.5555/3295222.3295408',
 '10.5555/3042817.3043083',
 '10.1145/3126686.3126723',
 '10.1109/tmm.2015.2407694',
 '10.1109/CVPR.2014.223',
 '10.1177/0278364913491297',
 '10.1007/978-3-642-33712-3_38',
 '10.1145/3123266.3123309',
 '10.5555/2002472.2002497']

In [47]:
%reload_ext autoreload
%autoreload 2
from cbf.cbf_hits import CBFH

In [50]:
CBFH('generative+adversarial+networks+audio+and+video').recommend()

init the CBF....
downloading paper meta data....
processing doi....


  0%|          | 0/280 [00:00<?, ?it/s]

extracting citation's doi....


  0%|          | 0/136 [00:00<?, ?it/s]

extracting citation's doi....


...

extracting citation's doi....


  0%|          | 0/40 [00:00<?, ?it/s]

extracting citation's doi....


  0%|          | 0/17 [00:00<?, ?it/s]

TypeError: 'NoneType' object does not support item assignment